>네이버 금융의 국내증시 거래상위에 있는 종목 100개의 일주일간 매분 시세 저장해주기
- 화요일에는 7일전까지 자료가 나왔는데 금요일밤 해보니 월요일부터 나온다. 언제 초기화될까?

In [1]:
import requests
import bs4
import pandas as pd
import datetime

In [2]:
url = "http://finance.naver.com/sise/sise_quant.nhn"

In [3]:
res = requests.get(url)

In [4]:
parsed = bs4.BeautifulSoup(res.text, "html.parser")

In [5]:
name_t = []
name_n = []
for td in parsed.find_all(name = "td"):
    if str(type(td.find("a"))) != "<class 'NoneType'>":
        name_t.append(td.find("a").text)
        name_n.append(td.find("a"))

In [6]:
address = []
for n in name_n:
    address.append(str(n)[43:49])
    
thistime = []
time = []
now = datetime.datetime.now()
for t in range(7):
    thistime.append("{:0=4}{:0=2}{:0=2}{:0=2}{:0=2}{:0=2}".format(now.year,now.month,
                                      now.day-6+t, now.hour+15,now.minute,now.second)) 
    time.append("{:0=4}-{:0=2}-{:0=2}".format(now.year,now.month,
                                      now.day-6+t))

page = []
for p in range(42):
    page.append(p+1)

In [7]:
url1 = "http://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}"

In [9]:
df_list = []
count = 0
for n in range(len(address[0:2])):
    for t in range(len(thistime[4:6])):
        data= []
        for p in page:
            res1 = requests.get(url1.format(address[n],thistime[t+2],p))
            parsed1 = bs4.BeautifulSoup(res1.text, "html.parser")
            for td in parsed1.find_all(name = "td"):
                if str(type(td.find("span"))) != "<class 'NoneType'>":
                    count = count +1
                    if count//7*7 == count - 1:
                        data.append(name_t[n])
                        data.append(time[t])
                    data.append(td.find("span").text)
                
        for i in range(4,len(data),9):
            data[i] = int(data[i])
        for j in range(0,len(data),9):
            df_list.append(data[j:j+9])

df = pd.DataFrame(df_list, columns= ["종목명", "날짜", "체결시각", "체결가", "전일비", 
                                     "매도", "매수", "거래량", "변동량"])